## 1. Enter the address, variable, and import the necessary library

In [ ]:
import dv_processing as dv
import torch
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import os
from matplotlib import pyplot as plt
import mmap
import aiofiles
import sys
from datetime import datetime
sys.path.append('./k_search_funciton')  # Add module path
import k_search_funciton.file_read as file_read

# Set Chinese font
try:
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei'] # or 'SimHei', 'Microsoft YaHei', etc.
    plt.rcParams['axes.unicode_minus'] = False  # Avoid negative signs displaying as squares
except Exception as e:
    print(f"Unable to set Chinese font: {e}. Chinese characters in charts may not display correctly.")


INPUT_FOLDER = "/mnt/f/raw2event/0511_1620/"  # Replace with your folder path
FILE_SUFFIX = "172631"  # Replace with your file suffix

try:
    # Find matching files
    files = file_read.find_matching_files(INPUT_FOLDER, FILE_SUFFIX)        
except FileNotFoundError as e:
    print(f"Error: {str(e)}")


PI_IMAGE_WEIGHT = 692  # Width of the image
PI_IMAGE_HEIGHT = 520  # Height of the image

## 2. Load data

In [ ]:
# Set file paths

# Load frame data
dv_frames, dv_frames_timestamps = file_read.load_frames(files['dv'])

print(f"DV frame data shape: {dv_frames.shape}")
print(f"Number of DV frame timestamps: {len(dv_frames_timestamps)}")

# Load event data
dv_events_tensor = file_read.load_events(files['dv'])
print(f"DV event data shape: {dv_events_tensor.shape}")
print(f"DV event timestamp range: [{dv_events_tensor[:, 0].min()}, {dv_events_tensor[:, 0].max()}]")

# Read PI camera data (asynchronous)
# pi_timestamps, real_timestamps, raw_frames, rgb_frames = await file_read.read_pi_data_async(
#     METADATA_INPUT,
#     RAW_FRAMES_INPUT,
#     RGB_FRAMES_INPUT,
#     PI_IMAGE_HEIGHT,
#     PI_IMAGE_WEIGHT
# )

rgb_frames = file_read.read_rgb_frames(files['rgb_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WEIGHT)
raw_frames = file_read.read_raw_frames(files['raw_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WEIGHT)
pi_timestamps, real_timestamps = file_read.read_metadata(files['metadata'])

# Print timestamps
print(f"Number of timestamps: {len(pi_timestamps)}")
print(f"Timestamp range: {pi_timestamps[0]:.2f} - {pi_timestamps[-1]:.2f}")
print(f"Number of real timestamps: {len(real_timestamps)}")
print(f"Real timestamp range: {real_timestamps[0]:.2f} - {real_timestamps[-1]:.2f}")

# Create a figure with two subplots
plt.figure(figsize=(15, 6))  # Adjust total figure size

# First subplot: RAW frame
plt.subplot(1, 2, 1)  # 1st of 1 row, 2 columns
plt.imshow(raw_frames[0].numpy(), cmap='gray')
plt.title('RAW Frame 0')
plt.axis('off')  # Turn off axis

# Second subplot: RGB frame
plt.subplot(1, 2, 2)  # 2nd of 1 row, 2 columns
plt.imshow(rgb_frames[0].numpy())
plt.title('RGB Frame 0')
plt.axis('off')  # Turn off axis

# Adjust spacing between subplots
plt.tight_layout()

# Display the figure
plt.show()

## 3. Timestamp alignment

In [ ]:
# Calculate time offset
time_offset = file_read.calculate_time_offset(pi_timestamps, real_timestamps)
print(f"Time offset: {time_offset} microseconds")

# Adjust DV frame timestamps
dv_frames_timestamps = dv_frames_timestamps - time_offset

# Adjust DV event timestamps
dv_events_tensor[:, 0] = dv_events_tensor[:, 0] - time_offset

# Print adjusted time ranges
print(f"Adjusted DV frame timestamp range: [{dv_frames_timestamps[0]}, {dv_frames_timestamps[-1]}]")
print(f"Adjusted DV event timestamp range: [{dv_events_tensor[:, 0].min()}, {dv_events_tensor[:, 0].max()}]")
print(f"PI timestamp range: {pi_timestamps[0]:.2f} - {pi_timestamps[-1]:.2f}")

## 4. Extract frame information based on frame data

In [ ]:
import k_search_funciton.tag_detector as tag_detector
import importlib
importlib.reload(tag_detector)
import cv2
TAG_REF_WIDTH = 287      # AprilTag reference width (pixels)
BARBARA_REF_SIZE = 861   # Barbara reference side length (pixels)
BARBARA_GAP = 82         # Gap between Barbara and tag (pixels)
margin_ratio = 0.03

# Create detector
detector = tag_detector.create_detector()

# Process RGB frame
barbara_info_rgb, cropped_rgb, ts_rgb = tag_detector.process_frame(
    rgb_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# Process RAW frame
barbara_info_raw, cropped_raw, ts_raw = tag_detector.process_frame(
    raw_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=True
)

# Process DV frame
barbara_info_dv, cropped_dv, ts_dv = tag_detector.process_frame(
    dv_frames[0].numpy(), dv_frames_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# Add these three lines before displaying images
def get_box_size(polygon):
    if polygon is None:
        return None
    width = np.linalg.norm(polygon[1] - polygon[0])  # Bottom edge length
    height = np.linalg.norm(polygon[2] - polygon[1])  # Right edge length
    return (width, height)

print(f"RGB box size: {get_box_size(barbara_info_rgb['polygon'] if barbara_info_rgb else None)}")
print(f"RAW box size: {get_box_size(barbara_info_raw['polygon'] if barbara_info_raw else None)}")
print(f"DV box size: {get_box_size(barbara_info_dv['polygon'] if barbara_info_dv else None)}")

# Display single frame cropping results
plt.figure(figsize=(12, 12))
# 1. RGB original image
plt.subplot(3, 2, 1)
frame_img_rgb = rgb_frames[0].numpy().copy()
plt.imshow(frame_img_rgb)
if barbara_info_rgb is not None:
    poly = barbara_info_rgb['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RGB Frame with Barbara Region')
plt.axis('off')

# 2. RGB cropped image
plt.subplot(3, 2, 2)
if cropped_rgb is not None:
    plt.imshow(cropped_rgb)
    plt.title('RGB Cropped Frame')
else:
    plt.title('RGB Cropped Frame (None)')
plt.axis('off')

# 3. RAW original image
plt.subplot(3, 2, 3)
frame_img_raw = raw_frames[0].numpy().copy()
plt.imshow(frame_img_raw, cmap='gray')
if barbara_info_raw is not None:
    poly = barbara_info_raw['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RAW Frame with Barbara Region')
plt.axis('off')

# 4. RAW cropped image
plt.subplot(3, 2, 4)
if cropped_raw is not None:
    plt.imshow(cropped_raw, cmap='gray')
    plt.title('RAW Cropped Frame')
else:
    plt.title('RAW Cropped Frame (None)')
plt.axis('off')

# 5. DV original image
plt.subplot(3, 2, 5)
frame_img_dv = dv_frames[0].numpy().copy()
plt.imshow(frame_img_dv, cmap='gray')
if barbara_info_dv is not None:
    poly = barbara_info_dv['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('DV Frame with Barbara Region')
plt.axis('off')

# 6. DV cropped image
plt.subplot(3, 2, 6)
if cropped_dv is not None:
    plt.imshow(cropped_dv, cmap='gray')
    plt.title('DV Cropped Frame')
else:
    plt.title('DV Cropped Frame (None)')
plt.axis('off')

plt.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.15)  # Reduce column spacing
plt.show()

In [ ]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import k_search_funciton.tag_detector as tag_detector
import importlib
importlib.reload(tag_detector)

n_workers = 4  # Number of threads
# Construct parameter lists
margin_ratios = [margin_ratio] * n_workers
tag_ref_widths = [TAG_REF_WIDTH] * n_workers
barbara_ref_sizes = [BARBARA_REF_SIZE] * n_workers
barbara_gaps = [BARBARA_GAP] * n_workers
is_raws_rgb = [False] * n_workers
is_raws_raw = [True] * n_workers

rgb_frame_batches = tag_detector.split_batches(rgb_frames.numpy(), n_workers)
raw_frame_batches = tag_detector.split_batches(raw_frames.numpy(), n_workers)
ts_batches = tag_detector.split_batches(pi_timestamps, n_workers)

dv_frame_batches = tag_detector.split_batches(dv_frames.numpy(), n_workers)
dv_ts_batches = tag_detector.split_batches(dv_frames_timestamps, n_workers)

from tqdm import tqdm

# Batch process RGB frames to get cropping box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=n_workers) as executor:
    rgb_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            rgb_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=n_workers
    ):
        rgb_all_results.append(batch_result)

# Merge all results
rgb_crops_info = [item for batch in rgb_all_results for item in batch]

# Sort by timestamp
rgb_crops_info.sort(key=lambda x: x[2])  # x[2] is timestamp

# Batch process RAW frames to get cropping box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=n_workers) as executor:
    raw_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            raw_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_raw
        ),
        total=n_workers
    ):
        raw_all_results.append(batch_result)

# Merge all results
raw_crops_info = [item for batch in raw_all_results for item in batch]

# Sort by timestamp
raw_crops_info.sort(key=lambda x: x[2])  # x[2] is timestamp

# Batch process DV frames to get cropping box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=n_workers) as executor:
    dv_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            dv_frame_batches, dv_ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=n_workers
    ):
        dv_all_results.append(batch_result)

# Merge all results
dv_crops_info = [item for batch in dv_all_results for item in batch]

# Sort by timestamp
dv_crops_info.sort(key=lambda x: x[2])  # x[2] is timestamp

# Column 0: barbara_info - Contains detected Barbara tag information, such as polygon vertices, rotation angle, center point, etc.
# Column 1: tag_info - Contains detected AprilTag information, such as ID, position, etc.
# Column 2: timestamp - Timestamp of the current frame

In [ ]:
plt.figure(figsize=(15, 10))

# Extract center point trajectories
rgb_centers = np.array([info[0]['center'] for info in rgb_crops_info if info[0] is not None])
raw_centers = np.array([info[0]['center'] for info in raw_crops_info if info[0] is not None])
dv_centers  = np.array([info[0]['center'] for info in dv_crops_info  if info[0] is not None])

# Extract angles
rgb_angles = np.array([info[0]['angle'] for info in rgb_crops_info if info[0] is not None])
raw_angles = np.array([info[0]['angle'] for info in raw_crops_info if info[0] is not None])
dv_angles  = np.array([info[0]['angle'] for info in dv_crops_info  if info[0] is not None])

rgb_times = [info[2] for info in rgb_crops_info if info[0] is not None]
raw_times = [info[2] for info in raw_crops_info if info[0] is not None]
dv_times  = [info[2] for info in dv_crops_info  if info[0] is not None]

plt.figure(figsize=(10, 8))

# 1. Combine trajectories in one plot
plt.subplot(2, 1, 1)
s = 1  # Point size
lw = 1  # Line width

if len(rgb_centers) > 0:
    plt.plot(rgb_centers[:,0], rgb_centers[:,1], '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_centers[:,0], rgb_centers[:,1], color='red', s=s)

if len(raw_centers) > 0:
    plt.plot(raw_centers[:,0], raw_centers[:,1], '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_centers[:,0], raw_centers[:,1], color='green', s=s)

if len(dv_centers) > 0:
    plt.plot(dv_centers[:,0], dv_centers[:,1], '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_centers[:,0], dv_centers[:,1], color='blue', s=s)

plt.title('Barbara Center Trajectories')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().invert_yaxis()
plt.axis('equal')
plt.grid(True)
plt.legend()

# 2. Combine angles over time in one plot
plt.subplot(2, 1, 2)
s = 4  # Point size

if len(rgb_angles) > 0:
    plt.plot(rgb_times, rgb_angles, '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_times, rgb_angles, color='red', s=s)

if len(raw_angles) > 0:
    plt.plot(raw_times, raw_angles, '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_times, raw_angles, color='green', s=s)

if len(dv_angles) > 0:
    plt.plot(dv_times, dv_angles, '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_times, dv_angles, color='blue', s=s)

plt.title('Barbara Angle vs Time')
plt.xlabel('Timestamp')
plt.ylabel('Angle (deg)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## 5. Filter event data

In [ ]:
import k_search_funciton.event_filter as event_filter
import importlib
importlib.reload(event_filter)
import math

# Assuming box_size is a tuple (w, h)
def round_up_to_10(x):
    return int(math.ceil(x / 10.0) * 10)

# Get the maximum side length of each box
rgb_box_size = max(barbara_info_rgb['polygon'].ptp(axis=0))
raw_box_size = max(barbara_info_raw['polygon'].ptp(axis=0))
dv_box_size = max(barbara_info_dv['polygon'].ptp(axis=0))

# Round up to the nearest multiple of 10
RGB_BOX_SIZE_FOR_EVENT = round_up_to_10(rgb_box_size)
RAW_BOX_SIZE_FOR_EVENT = round_up_to_10(raw_box_size)
DV_BOX_SIZE_FOR_EVENT = round_up_to_10(dv_box_size)
BATCH_SIZE_FOR_EVENT = 100000
N_WORKERS_FOR_EVENT = 4

# DV event filtering
# Use parallel processing function
filtered_events_dv = event_filter.filter_events_parallel(
    events_tensor=dv_events_tensor,  # Your event data
    crops_info=dv_crops_info,        # Cropping box information
    target_size=DV_BOX_SIZE_FOR_EVENT,  # Target cropping box size
    transform=True,                    # Whether to transform coordinates
    batch_size=BATCH_SIZE_FOR_EVENT,                # Number of events per batch
    n_workers=N_WORKERS_FOR_EVENT                       # Number of parallel processes
)

# Check results
print(f"Number of filtered DV events: {len(filtered_events_dv)}")
print(f"DV event timestamp range: [{filtered_events_dv[0, 0]}, {filtered_events_dv[-1, 0]}]")

## 6. Event Data Visualization

In [ ]:
from IPython.display import display, clear_output
import datetime 

def play_filtered_events(events, interval_ms=33, window_size=(230, 230)):
    """
    Play filtered event data
    
    Parameters:
    events: Filtered event data (PyTorch tensor)
    interval_ms: Time interval between frames (milliseconds)
    window_size: Display window size
    """
    # Create event accumulator
    accumulator = dv.Accumulator(window_size)
    
    # Set accumulator parameters
    accumulator.setEventContribution(0.25)
    accumulator.setNeutralPotential(0.5)
    accumulator.setMinPotential(0.0)
    accumulator.setMaxPotential(1.0)
    accumulator.setDecayFunction(dv.Accumulator.Decay.LINEAR)
    accumulator.setDecayParam(1e-6)
    accumulator.setSynchronousDecay(False)
    accumulator.setIgnorePolarity(False)
    
    # Create preview window
    cv2.namedWindow("Events Preview", cv2.WINDOW_NORMAL)
    
    # Create event slicer
    slicer = dv.EventStreamSlicer()
    
    # Frame counter
    frame_counter = 0
    
    def accumulate_events(event_slice):
        nonlocal frame_counter
        
        # Pass event slice to accumulator
        accumulator.accept(event_slice)
        
        # Generate frame
        frame = accumulator.generateFrame()
        
        # Increment frame counter
        frame_counter += 1
        
        # Display frame
        cv2.imshow("Events Preview", frame.image)
        cv2.waitKey(2)
    
    # Set time interval
    slicer.doEveryTimeInterval(datetime.timedelta(milliseconds=interval_ms), accumulate_events)
    
    print("Starting event data playback...")
    
    # Get event data
    # We need to convert PyTorch tensor to event format that dv library can process
    # Process events in batches, 1000 events per batch
    batch_size = 1000
    total_events = len(events)
    
    for i in range(0, total_events, batch_size):
        # Get current batch of events
        batch_events = events[i:min(i+batch_size, total_events)]
        
        # Convert to format that dv library can process
        batch = dv.EventStore()
        for j in range(len(batch_events)):
            event = batch_events[j]
            x = int(event[0].item())
            y = int(event[1].item())
            timestamp = int(event[2].item())
            polarity = int(event[3].item())
            batch.push_back(dv.Event(x, y, timestamp, polarity))
        
        # Pass events to slicer
        slicer.accept(batch)
        
        # Check if 'q' key is pressed to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    print(f"Total frames: {frame_counter}")
    print("Event playback completed")
    cv2.destroyAllWindows()


# Play events
play_filtered_events(filtered_events_dv, 33, (DV_BOX_SIZE_FOR_EVENT, DV_BOX_SIZE_FOR_EVENT))

In [ ]:
import k_search_funciton.interval_fit as interval_fit
import importlib
importlib.reload(interval_fit)

num_pixels_dv, dt_dv, mu_dv, sigma_dv = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_dv,
    min_events_per_pixel=10,         # For example, at least 10 events per pixel
    max_dt_us_for_plot=100000,          # For example, only plot intervals below 200 microseconds
    plot_bins=100,
    type='DV'
)
plt.show()  # Display the plot